In [1]:
import logging

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import warnings
import os

# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None

import azureml.core
from azureml.core import Experiment, Workspace, Dataset
from azureml.train.automl import AutoMLConfig
from datetime import datetime

In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-130297")
exp = Experiment(workspace=ws, name="hypd_model")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-130297
Azure region: southcentralus
Subscription id: 2552278b-2817-43a7-820e-5a5a53ff9e19
Resource group: aml-quickstarts-130297


In [3]:
run = exp.start_logging()

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
cluster_name="hd-cluster"
try:
  compute_target=ComputeTarget(workspace=ws,name=cluster_name)
  print("Found existing compute target")
except ComputeTargetException:
  print('Creating a new compute target')
  compute_config=AmlCompute.provisioning_configuration(vm_size='Standard_DS12_V2',max_nodes=4)
  compute_target=ComputeTarget.create(ws,cluster_name,compute_config)
  

compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

Creating a new compute target
hd-cluster AmlCompute Creating
ds12 AmlCompute Succeeded
sds12 ComputeInstance Succeeded
AUTOM-ML-cluster AmlCompute Succeeded


In [8]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os
from azureml.core import ScriptRunConfig, Experiment


# Specify parameter sampler
ps = RandomParameterSampling(parameter_space={"--C": choice(0.001,0.01, 0.1, 1, 10, 100) , "--coef0": choice(0,1,2,3)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=5,slack_factor=0.1)
if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est =ScriptRunConfig(source_directory='./',compute_target=compute_target,script='train.py') ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =HyperDriveConfig(estimator=est , hyperparameter_sampling=ps, policy=policy , primary_metric_name='Accuracy',primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,max_total_runs=50,max_concurrent_runs=4)

In [9]:
# Submit your experiment
hyperdrive_run=exp.submit(hyperdrive_config)

AttributeError: 'ScriptRunConfig' object has no attribute '_compute_target'